In [1]:
from z3 import *
import numpy as np

In [2]:
width = 8
dims = ((3, 3),
            (3, 5),
            (5, 3),
            (5, 5))
print(dims)
max_height = int(np.sum([dims[i][1] for i in range(len(dims))]))
print(f"Maximum height reachable: {max_height}")

((3, 3), (3, 5), (5, 3), (5, 5))
Maximum height reachable: 16


In [3]:
def min_height(dims):
  return int(np.min([dims[i][1] for i in range(len(dims))]))

def compute_area_circuits(dims):
  return sum([dim[0]* dim[1] for dim in dims])


def vlsi_instance(dims, width, max_height):
        """
                Variable declaration
        """
        height = Int('height')
        corner_coords = [[Int("corner_coords_%s_%s" % (i + 1, j + 1)) for j in range(2)] for i in range(len(dims))]

        # The additional Boolean variable rot is used to indicate the change of orientation of the block
        # Since they are all rectangles, a simple swap between the two edges is enough to distinguish between
        # the original rectangle and the rotated one
        # z = False - the block is in its original orientation
        # z = True - the block is rotated by 90 degrees
        rot = [Bool(f"rot_{i + 1}") for i in range(len(dims))]

        """
                Constraint definition
        """
        # board boundaries constraints
        boundaries_c_x = [And(0 <= corner_coords[i][0],
                                corner_coords[i][0] + rot[i] * dims[i][0] + Not(rot[i]) * dims[i][1] <= width) for i in range(len(dims))]
        boundaries_c_y = [And(0 <= corner_coords[i][1],
                                corner_coords[i][1] + rot[i] * dims[i][0] + Not(rot[i]) * dims[i][1] <= height) for i in range(len(dims))]
        height_constraint = [And(height < max_height, height > min_height(dims))]

        # no overlapping constraints


        no_overlapping = []
        for i in range(len(dims)):
                for j in range(len(dims)): # to fix
                        if i != j:
                                no_overlapping.append(
                                # block i is on the left of block j: xi < xj
                                If(corner_coords[i][0] < corner_coords[j][0],
                                        corner_coords[i][0] + (rot[i] * dims[i][0]) + (Not(rot[i]) * dims[i][1]) <= corner_coords[j][0],
                                # block i is below block j: yi < yj
                                If(corner_coords[i][1] < corner_coords[j][1],
                                        corner_coords[i][1] + (rot[i] * dims[i][0]) + (Not(rot[i]) * dims[i][1]) <= corner_coords[j][1],
                                # block i is on the right of block j: xi > xj
                                If(corner_coords[i][0] > corner_coords[j][0],
                                        corner_coords[i][0] - (rot[j] * dims[j][0]) - (Not(rot[j]) * dims[j][1]) >= corner_coords[j][0],
                                # block i is above block j: yi > yj
                                If(corner_coords[i][1] > corner_coords[j][1],
                                        corner_coords[i][1] - (rot[j] * dims[j][0]) - (Not(rot[j]) * dims[j][1]) >= corner_coords[j][1],
                                        False)))))

        #     print(no_overlapping)
        all_constraints = height_constraint + boundaries_c_x  + boundaries_c_y + no_overlapping

        """
                Solving phase
        """
        solver = Optimize()
        solver.add(all_constraints)
        # minimizing function
        circuit_area = compute_area_circuits(dims)
        solver.minimize(width* height - circuit_area)
        if solver.check() == sat:
                m = solver.model()
                return m.evaluate(height), [[int(m.evaluate(corner_coords[i][j]).as_string()) for j in range(2)] for i in range(len(dims))]
        else:
                print("Failed to find a solution")
                return None


In [4]:
vlsi_instance(dims, width, max_height)

(8, [[5, 0], [5, 4], [0, 0], [0, 3]])